In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from pygam import LinearGAM, s, f
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
import feature_engine.imputation as imp
from feature_engine.encoding import OneHotEncoder

import matplotlib.pyplot as plt

In [3]:
X_train = pd.read_csv("Xtrain_feature_sel.csv")
X_test = pd.read_csv("Xtest_feature_sel.csv")
ytrain = pd.read_csv("ytrain.csv")
ytest = pd.read_csv("ytest.csv")
print("Shape of X Train: {}".format(X_train.shape))
print("Shape of X Test: {}".format(X_test.shape))
print("Shape of y Train: {}".format(ytrain.shape))
print("Shape of y Test: {}".format(ytest.shape))

Shape of X Train: (8672, 17)
Shape of X Test: (2168, 17)
Shape of y Train: (8672, 1)
Shape of y Test: (2169, 1)


In [5]:
# types of the variables
X_train.dtypes

Reviews                    float64
Rating                     float64
Price                      float64
Last Updated                object
Category_FAMILY              int64
Category_GAME                int64
Category_TOOLS               int64
Size_Varies with device      int64
Size_0.1-10MB                int64
Size_10.1-20MB               int64
Size_30.1-40MB               int64
Size_20.1-30MB               int64
Type_Free                    int64
Content Rating_Everyone      int64
Content Rating_Teen          int64
Genres_Entertainment         int64
Genres_Education             int64
dtype: object

In [11]:
ytrain

,Installs
0,"1,000,000+"
1,"50,000+"
2,100+
3,"10,000+"
4,"5,000,000+"
...,...
8667,"10,000,000+"
8668,"10,000,000+"
8669,"10,000,000+"
8670,"100,000+"


In [8]:
# create linear regression
regr = linear_model.LinearRegression()
# train model using training set
reg = regr.fit(X_train[['Size_Varies with device']],ytrain)

ValueError: could not convert string to float: '1,000,000+'